### SLICER_FITS

Para cortar los cubos fits de PANIC.

**inputs**

1. Carpeta input con la imagenes fits originales
2. El nombre de la serie (lo pide el programa)

Se crean fits individuales con el header del cubo original. 
El nombre de los fits individuales es *'nombre de la serie'_i*
Se guardan en output/serie/originals


In [16]:
from astropy.io import fits
from astropy.io.fits import Header
import os

path = os.getcwd()

# function to make new directories in the output dir, or to use the existing one
def make_dir(name):
    path_new_dir = os.path.dirname(path)+f'/output/{name}'
    if not os.path.exists(path_new_dir):
        os.makedirs(path_new_dir)
        directory = path_new_dir
    else:
        directory = path_new_dir
    return directory

#

#~~~~  More than one cube at a time ~~~~#

#object = input('Objeto de la FocusSerie: ')

#total_files = [file for file in os.listdir(os.path.dirname(path)+'/input/') if file.startswith(f'FocusSeries_{object}')]
#for i in range(len(total_files)):
#    focusSerie = f'{object}_{i+1:04d}'
#    # open fit cube
#    hdul = fits.open(os.path.dirname(path)+f'/input/FocusSeries_{focusSerie}.fits')
#    hdr = hdul[0].header
#    img = hdul[0].data
#    directory = make_dir(focusSerie)
#
#    # slice and save in different fits
#    for slice_index in range(len(img)):
#        img_ind = img[slice_index, :, :]
#        hdr_ind = Header(cards=hdr.cards)
#        slc = fits.PrimaryHDU(data=img_ind, header=hdr_ind)
#        slc.writeto(os.path.dirname(path) + f'/output/{focusSerie}/{focusSerie}_slice_{slice_index+1}.fits', overwrite=True)



#~~~~     One cube at a time     ~~~~#

focusSerie = input('Focus Serie (ej: M34_0004): ')

hdul = fits.open(os.path.dirname(path)+f'/input/FocusSeries_{focusSerie}.fits')
hdr = hdul[0].header
img = hdul[0].data
dir_focusSerie= make_dir(focusSerie)
dir_originals = make_dir(f'{focusSerie}/originals')

# slice and save in different fits
for slice_index in range(len(img)):
    img_ind = img[slice_index, :, :]
    hdr_ind = Header(cards=hdr.cards)
    slc = fits.PrimaryHDU(data=img_ind, header=hdr_ind)
    slc.writeto(os.path.dirname(path) + f'/output/{focusSerie}/originals/{focusSerie}_{slice_index+1}.fits', overwrite=True)



Objeto de la FocusSerie:M34
Ejecution time: 8.214513063430786


#### Comprobación del código

Ver si las imágenes son iguales a la correspondiente en el cubo

In [46]:
fit = fits.open(os.path.dirname(path)+f'/input/FocusSeries_M34_0015.fits')
img = fit[0].data
hdr = fit[0].header

for i in range(len(img)):
    fit_i = fits.open(os.path.dirname(path)+f'/output/M34_0015/M34_0015_slice_{i+1}.fits')
    hdr_i = fit_i[0].header
    img_i = fit_i[0].data

    print((img_i == img[i,:,:]).all())
    

True
True
True
True
True
True
True
True
True
True


Ver si el header es igual, exceptuando las primeras filas. 
Se eliminan al cortar:
1. NAXIS3
2. EXTEND
3. BSCALE
4. BZERO

In [44]:
hdr_i = fits.open(os.path.dirname(path)+f'/output/M34_0015/M34_0015_slice_1.fits')[0].header
hdr_i

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                   32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 4096                                                  
NAXIS2  =                 4096                                                  
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
BUNIT   = 'adu     '           / [adu]                                          
MJD-OBS =         59975.825725 / [d] Modified Julian Date of DATE-OBS           
DATE-OBS= '2023-01-31T19:47:53.4321' / [d] UTC end of first frame read          
DATE    = '2023-01-31T19:49:09' / file creation date (YYYY-MM-DDThh:mm:ss UT)   
UT      =          71273.43212 / [s] 19:47:53.4321 UTC at end of first frame rea
LST     =         15706.8073

In [47]:
hdr_i[7:] == hdr[11:]

True